In [ ]:
!pip -q install ccxt pandas matplotlib ta pytz ipywidgets

In [ ]:
import ccxt, pandas as pd
exchange = ccxt.kraken({'enableRateLimit': True})
exchange.load_markets()
market = 'BTC/USD' if 'BTC/USD' in exchange.symbols else 'XBT/USD'

In [ ]:
import pandas as pd

def fetch_ohlcv(timeframe='1h', limit=500):
    ohlcv = exchange.fetch_ohlcv(market, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['time','open','high','low','close','volume'])
    df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True).dt.tz_convert('America/Denver')
    df.set_index('time', inplace=True)
    return df

def add_indicators(df):
    df = df.copy()
    df['SMA50'] = df['close'].rolling(50).mean()
    df['SMA200'] = df['close'].rolling(200).mean()
    delta = df['close'].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = (-delta.clip(upper=0)).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def generate_signal(df):
    row = df.iloc[-1]
    if row['SMA50'] > row['SMA200'] and row['RSI'] >= 50: return 'Bullish'
    if row['SMA50'] < row['SMA200'] and row['RSI'] <= 50: return 'Bearish'
    return 'Neutral'

def run_once(timeframe='1h'):
    df = add_indicators(fetch_ohlcv(timeframe))
    sig = generate_signal(df)
    now = pd.Timestamp.now(tz='America/Denver').strftime('%Y-%m-%d %H:%M:%S')
    print(f'{now} — {sig}  |  close={df.close.iloc[-1]:,.2f}')
    return df

In [ ]:
import threading, time
from IPython.display import clear_output, display

_live = {'run': False, 'thread': None, 'timeframe': '1h', 'interval_s': 60}
_latest_df = None
_lock = threading.Lock()

def _live_worker():
    global _latest_df
    while _live['run']:
        try:
            df = run_once(_live['timeframe'])
            with _lock:
                _latest_df = df
        except Exception as e:
            print(f'[live] error: {e}')
        time.sleep(_live['interval_s'])

def start_live(timeframe='1h', interval_s=60):
    if _live['run']:
        print('Live already running.')
        return
    _live.update({'run': True, 'timeframe': timeframe, 'interval_s': interval_s})
    _live['thread'] = threading.Thread(target=_live_worker, daemon=True)
    _live['thread'].start()
    print(f'Started live mode: timeframe={timeframe}, every {interval_s}s (background thread).')

def stop_live():
    if not _live['run']:
        print('Live not running.')
        return
    _live['run'] = False
    print('Stopping live…')

In [ ]:
import matplotlib.pyplot as plt

def show_live_chart(refresh_s=10):
    """
    Call this in its own cell; it refreshes while live mode is running.
    Stop it by interrupting the cell or calling stop_live().
    """
    while _live['run']:
        with _lock:
            df = None if _latest_df is None else _latest_df.copy()
        if df is not None and len(df) > 0:
            clear_output(wait=True)
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
            ax1.plot(df.index, df['close'], label='Price')
            ax1.plot(df.index, df['SMA50'], label='SMA50')
            ax1.plot(df.index, df['SMA200'], label='SMA200')
            ax1.set_title('BTC with SMA50/200')
            ax1.legend()
            ax2.plot(df.index, df['RSI'], label='RSI')
            ax2.axhline(30, linestyle='--')
            ax2.axhline(70, linestyle='--')
            ax2.set_title('RSI')
            plt.show()
        time.sleep(refresh_s)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

tf_dd = widgets.Dropdown(options=['5m','15m','1h','4h','1d'], value='1h', description='Timeframe:')
snap_btn = widgets.Button(description='Run snapshot')
start_btn = widgets.Button(description='Start live')
stop_btn = widgets.Button(description='Stop live')
live_chart_btn = widgets.Button(description='Open live chart')

out = widgets.Output()

def on_snapshot(_):
    with out:
        print('Running snapshot…')
        df = run_once(tf_dd.value)
        print(df.tail(3)[['close','SMA50','SMA200','RSI']])

def on_start(_):
    with out:
        start_live(timeframe=tf_dd.value, interval_s=60)

def on_stop(_):
    with out:
        stop_live()

def on_live_chart(_):
    with out:
        print('Opening live chart (interrupt cell or press Stop live to end)…')
    show_live_chart(refresh_s=15)

snap_btn.on_click(on_snapshot)
start_btn.on_click(on_start)
stop_btn.on_click(on_stop)
live_chart_btn.on_click(on_live_chart)

display(tf_dd, widgets.HBox([snap_btn, start_btn, stop_btn, live_chart_btn]), out)

* Run **Run snapshot** for a one-time check.
* Click **Start live** to begin background updates (non-blocking).
* Click **Open live chart**; it will refresh until you interrupt that cell or click **Stop live**.
* Click **Stop live** when you’re done.